In [ ]:
def with_retry(
    max_retries: int = 3,
    retry_delay: float = 1.0,
    max_backoff: float = 60.0,
    retry_on: List[type] = [RateLimitError, aiohttp.ClientError, asyncio.TimeoutError]
):
    """
    Decorator to automatically retry functions on specified exceptions
    
    Args:
        max_retries: Maximum number of retry attempts
        retry_delay: Initial delay for retries (seconds)
        max_backoff: Maximum backoff time (seconds)
        retry_on: List of exception types to retry on
        
    Returns:
        Decorated function with retry logic
    """
    def decorator(func: Callable[..., T]) -> Callable[..., T]:
        @functools.wraps(func)
        async def wrapper(*args, **kwargs) -> T:
            last_exception = None
            
            for attempt in range(max_retries + 1):
                try:
                    return await func(*args, **kwargs)
                except tuple(retry_on) as e:
                    last_exception = e
                    
                    if attempt == max_retries:
                        # Log and raise on final attempt
                        if isinstance(e, RateLimitError):
                            logger.error(f"Rate limit exceeded, max retries reached")
                        else:
                            logger.error(f"Request failed after {max_retries} retries: {e}")
                        break
                    
                    # Calculate backoff time with exponential backoff and jitter
                    backoff = min(
                        max_backoff,
                        retry_delay * (2 ** attempt) * (0.9 + 0.2 * random.random())
                    )
                    
                    if isinstance(e, RateLimitError) and hasattr(e, 'retry_after') and e.retry_after:
                        # Use the server-suggested retry time if available
                        try:
                            retry_time = float(e.retry_after)
                            backoff = min(max_backoff, retry_time)
                        except (ValueError, TypeError):
                            pass
                    
                    logger.warning(f"Attempt {attempt+1}/{max_retries+1} failed: {e}. Retrying in {backoff:.2f}s")
                    await asyncio.sleep(backoff)
            
            # If we get here, all retries failed
            if last_exception:
                raise last_exception
            raise RuntimeError("All retries failed with unknown error")
            
        return wrapper
    return decorator


def with_logging(name: Optional[str] = None):
    """
    Decorator to add logging to request methods
    
    Args:
        name: Optional name for the log messages
        
    Returns:
        Decorated function with logging
    """
    def decorator(func: Callable[..., T]) -> Callable[..., T]:
        @functools.wraps(func)
        async def wrapper(*args, **kwargs) -> T:
            func_name = name or func.__name__
            
            # Extract URL from args/kwargs based on common function signatures
            url = None
            if len(args) > 1 and isinstance(args[1], str):
                url = args[1]
            elif 'url' in kwargs:
                url = kwargs['url']
            elif 'endpoint' in kwargs:
                url = kwargs['endpoint']
            
            log_suffix = f" to {url}" if url else ""
            logger.debug(f"Making {func_name}{log_suffix}")
            
            try:
                result = await func(*args, **kwargs)
                logger.debug(f"{func_name} successful{log_suffix}")
                return result
            except Exception as e:
                logger.error(f"{func_name} failed{log_suffix}: {e}")
                raise
                
        return wrapper
    return decorator


# ----- Request Handler -----

class RequestHandler:
    """
    Handles HTTP requests with proper error handling, retries, and resource management
    
    Features:
    - Async context manager support
    - Automatic retries with exponential backoff
    - Flexible request methods with proper error handling
    - Intelligent response parsing
    - Resource management
    """
    
    def __init__(self, 
                 base_url: str, 
                 auth_token: Optional[str] = None, 
                 default_headers: Optional[Dict[str, str]] = None,
                 timeout: float = 30.0,
                 cookies: Optional[Dict[str, str]] = None):
        """
        Initialize RequestHandler
        
        Args:
            base_url: Base URL for API requests
            auth_token: Authentication token (optional)
            default_headers: Default headers to include in all requests
            timeout: Request timeout in seconds
            cookies: Cookies to include in all requests
        """
        self.base_url = base_url.rstrip('/')
        self.auth_token = auth_token
        self.default_headers = default_headers or {}
        self.timeout = timeout
        self.cookies = cookies or {}
        self.session: Optional[aiohttp.ClientSession] = None
        
        # Add auth token to cookies if provided
        if auth_token:
            self.cookies["GSSSO"] = auth_token
    
    async def __aenter__(self) -> 'RequestHandler':
        """Initialize session when entering context"""
        await self._ensure_session()
        return self
    
    async def __aexit__(self, exc_type, exc_val, exc_tb) -> None:
        """Close session when exiting context"""
        await self.close()
    
    async def _ensure_session(self) -> aiohttp.ClientSession:
        """Ensure a valid session exists and return it"""
        if self.session is None or self.session.closed:
            self.session = aiohttp.ClientSession(
                cookies=self.cookies,
                timeout=aiohttp.ClientTimeout(total=self.timeout),
                headers=self.default_headers
            )
        return self.session
    
    def _build_url(self, endpoint: str) -> str:
        """Build full URL from endpoint"""
        if endpoint.startswith(('http://', 'https://')):
            # If endpoint is already a full URL, return it as is
            return endpoint
        # Otherwise, join with base URL
        return urljoin(f"{self.base_url}/", endpoint.lstrip('/'))
    
    def _merge_headers(self, headers: Optional[Dict[str, str]] = None) -> Dict[str, str]:
        """Merge default headers with request-specific headers"""
        result = self.default_headers.copy()
        if headers:
            result.update(headers)
        return result
    
    async def _check_response(self, response: aiohttp.ClientResponse) -> None:
        """
        Check response for errors
        
        Args:
            response: API response
            
        Raises:
            RateLimitError: If rate limit is exceeded
            APIError: For other error status codes
            AuthError: For authentication errors
        """
        if response.status == 429:
            retry_after = response.headers.get("Retry-After")
            raise RateLimitError("Rate limit exceeded", retry_after)
            
        if response.status == 401:
            raise AuthError("Authentication failed")
            
        if response.status == 403:
            raise AuthError("Permission denied")
            
        if response.status == 404:
            raise APIError("Resource not found", response.status)
            
        if response.status >= 400:
            error_text = await response.text()
            raise APIError(f"Request failed: {error_text[:200]}", response.status)
    
    @with_logging("GET")
    @with_retry()
    async def get(self, 
                 endpoint: str, 
                 params: Optional[Dict[str, Any]] = None, 
                 headers: Optional[Dict[str, str]] = None) -> str:
        """
        Make a GET request
        
        Args:
            endpoint: API endpoint or full URL
            params: Query parameters
            headers: Additional headers
            
        Returns:
            Response text
        """
        url = self._build_url(endpoint)
        session = await self._ensure_session()
        
        try:
            async with session.get(
                url, 
                params=params, 
                headers=self._merge_headers(headers)
            ) as response:
                await self._check_response(response)
                return await response.text()
        except aiohttp.ClientError as e:
            raise ContentFetchError(f"Request failed: {str(e)}", url)
    
    @with_logging("POST")
    @with_retry()
    async def post(self, 
                  endpoint: str, 
                  data: Any = None, 
                  json_data: Any = None,
                  headers: Optional[Dict[str, str]] = None) -> str:
        """
        Make a POST request
        
        Args:
            endpoint: API endpoint or full URL
            data: Request data
            json_data: JSON request data (will be encoded)
            headers: Additional headers
            
        Returns:
            Response text
        """
        url = self._build_url(endpoint)
        session = await self._ensure_session()
        
        try:
            async with session.post(
                url, 
                data=data,
                json=json_data,
                headers=self._merge_headers(headers)
            ) as response:
                await self._check_response(response)
                return await response.text()
        except aiohttp.ClientError as e:
            raise ContentFetchError(f"Request failed: {str(e)}", url)
    
    @with_logging("PUT")
    @with_retry()
    async def put(self, 
                 endpoint: str, 
                 data: Any = None, 
                 json_data: Any = None,
                 headers: Optional[Dict[str, str]] = None) -> str:
        """
        Make a PUT request
        
        Args:
            endpoint: API endpoint or full URL
            data: Request data
            json_data: JSON request data (will be encoded)
            headers: Additional headers
            
        Returns:
            Response text
        """
        url = self._build_url(endpoint)
        session = await self._ensure_session()
        
        try:
            async with session.put(
                url, 
                data=data,
                json=json_data,
                headers=self._merge_headers(headers)
            ) as response:
                await self._check_response(response)
                return await response.text()
        except aiohttp.ClientError as e:
            raise ContentFetchError(f"Request failed: {str(e)}", url)
    
    @with_logging("DELETE")
    @with_retry()
    async def delete(self, 
                    endpoint: str, 
                    params: Optional[Dict[str, Any]] = None, 
                    headers: Optional[Dict[str, str]] = None) -> str:
        """
        Make a DELETE request
        
        Args:
            endpoint: API endpoint or full URL
            params: Query parameters
            headers: Additional headers
            
        Returns:
            Response text
        """
        url = self._build_url(endpoint)
        session = await self._ensure_session()
        
        try:
            async with session.delete(
                url, 
                params=params, 
                headers=self._merge_headers(headers)
            ) as response:
                await self._check_response(response)
                return await response.text()
        except aiohttp.ClientError as e:
            raise ContentFetchError(f"Request failed: {str(e)}", url)
    
    async def get_json(self, 
                      endpoint: str, 
                      params: Optional[Dict[str, Any]] = None, 
                      headers: Optional[Dict[str, str]] = None) -> Any:
        """
        Make a GET request and parse JSON response
        
        Args:
            endpoint: API endpoint or full URL
            params: Query parameters
            headers: Additional headers
            
        Returns:
            Parsed JSON response
        """
        response_text = await self.get(endpoint, params, headers)
        try:
            return json.loads(response_text)
        except json.JSONDecodeError as e:
            raise ContentFetchError(f"Invalid JSON response: {str(e)}", endpoint)
    
    async def post_json(self, 
                       endpoint: str, 
                       data: Any = None,
                       headers: Optional[Dict[str, str]] = None) -> Any:
        """
        Make a POST request with JSON data and parse JSON response
        
        Args:
            endpoint: API endpoint or full URL
            data: JSON data to send
            headers: Additional headers
            
        Returns:
            Parsed JSON response
        """
        response_text = await self.post(endpoint, json_data=data, headers=headers)
        try:
            return json.loads(response_text)
        except json.JSONDecodeError as e:
            raise ContentFetchError(f"Invalid JSON response: {str(e)}", endpoint)
    
    async def stream(self, 
                    endpoint: str, 
                    params: Optional[Dict[str, Any]] = None, 
                    headers: Optional[Dict[str, str]] = None) -> aiohttp.ClientResponse:
        """
        Make a streaming GET request
        
        Args:
            endpoint: API endpoint or full URL
            params: Query parameters
            headers: Additional headers
            
        Returns:
            Response object for streaming
        """
        url = self._build_url(endpoint)
        session = await self._ensure_session()
        
        try:
            response = await session.get(
                url, 
                params=params, 
                headers=self._merge_headers(headers)
            )
            await self._check_response(response)
            return response
        except aiohttp.ClientError as e:
            raise ContentFetchError(f"Request failed: {str(e)}", url)
    
    async def close(self) -> None:
        """Close the session and release resources"""
        if self.session and not self.session.closed:
            await self.session.close()
            self.session = None